In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bitcoin-tweets:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1155801%2F5141688%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240412%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240412T174658Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9eb96130a92d4440796a28d34d22a90cb6fd52d7c877e6c6f7fc8181cd4a95931445143553dfb84101533a354f9f094b38a5a8ce14a24300d3761755c6afc9c28a849b8b5ce965ffe410ab9cd8f930f037c439802096e6dd67b560fe5ce24c1704ccd1e76b31ca482b49a8b8151494e7f667af0c47a85f292347c418d3cfebb368f484e6739bd7fcdfbcaf568266dad020f54648d51c36b86e4dcfda79ce6cb7fcf4fc35026d2c6fd9747e60fcdf9f6bac7f1be806c9b342ecad232e3a836a4d91503fb6a057cde75193dfa0c1205dad3fac41149e5df6f0167daa84b6a97bdb1f90e400deb450e337510dd5dc46a3574c7b5650d0ca30904a45cb4f943cc6ed,project-dataset-dl:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4748637%2F8052119%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240412%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240412T174659Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9da4062fe023b87e2340998f211b0d00983779a269939813eb8b686e66f1ac904601a7d82fd37c89cb01a571b8f33a358836cc301ba9546305eb4cdc4d873067110130d76162085a87d0f83efbb8a30e2e35593e37cea41d628d4c0ee117935c268b79542b066ad46775400eb50e25ffb32bfef4e3cd01811c6c1637a3904c91110f4f8e013105e0806f4c885aa6596b57739dd4959df0ffa38789376e0963c362bfd8b5e868663d786937e52ad12c195a5211af5f71e946f0d29220d3362d5c4a98ab763243337f0832a4923c4b7e26229ba78f9f7a6c64c3f5bd64e05472f2b842180eff7ef5aaf31b6d7c6fa5280073aeffff0c87d80167ea8141fac909da'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 728456658 bytes downloaded
Downloaded and uncompressed: bitcoin-tweets
[==================================================] 3302829 bytes downloaded
Downloaded and uncompressed: project-dataset-dl
Data source import complete.


**Importing necessary libraries**

In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch

**Importing dataset**

In [ ]:
df=pd.read_csv('/kaggle/input/project-dataset-dl/Data_DL.csv', encoding = "ISO-8859-1")

In [ ]:
df

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,2016-06-27,0,Barclays and RBS shares suspended from trading...,Pope says Church should ask forgiveness from g...,Poland 'shocked' by xenophobic abuse of Poles ...,"There will be no second referendum, cabinet ag...","Scotland welcome to join EU, Merkel ally says",Sterling dips below Friday's 31-year low amid ...,No negative news about South African President...,Surge in Hate Crimes in the U.K. Following U.K...,...,German lawyers to probe Erdogan over alleged w...,"Boris Johnson says the UK will continue to ""in...",Richard Branson is calling on the UK governmen...,Turkey 'sorry for downing Russian jet',Edward Snowden lawyer vows new push for pardon...,Brexit opinion poll reveals majority don't wan...,"Conservative MP Leave Campaigner: ""The leave c...","Economists predict UK recession, further weake...","New EU 'superstate plan by Franc

**Creating Custom Dataset**

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


class NewsDataset(Dataset):
    def __init__(self, dataframe, max_length):
        self.data = dataframe
        self.max_length = max_length
        self.tokenizer = lambda x: x.split()  # Simple tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        news_item = self.data.iloc[index]
        text = news_item['Top1'] + ' ' + news_item['Top2']  # Combine Text1 and Text2
        label = news_item['Label']

        tokens = self.tokenizer(text)
        input_ids, attention_mask = self.tokenize_and_pad(tokens)

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'label': torch.tensor(label, dtype=torch.long)
        }

    def tokenize_and_pad(self, tokens):
        input_ids = [self.token_to_id(token) for token in tokens]
        attention_mask = [1] * len(input_ids)

        # Padding
        padding_length = self.max_length - len(input_ids)
        input_ids = input_ids + [0] * padding_length
        attention_mask = attention_mask + [0] * padding_length

        return input_ids, attention_mask

    def token_to_id(self, token):
        return hash(token) % 10000  # Hash function to convert token to ID (example)

max_length = 128
dataset = NewsDataset(df, max_length)

tokenizer = lambda x: x.split()  # Simple tokenizer
max_length = 128
dataset = NewsDataset(df, max_length)


**Creating a classifier**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(SimpleSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.fc1 = nn.Linear(embed_dim, 64)
        self.fc2 = nn.Linear(64, num_class)

    def forward(self, x, attention_mask):
        x = self.embedding(x)
        x = torch.mean(x * attention_mask.unsqueeze(-1), dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Assuming a vocabulary size and embedding dimension
vocab_size = 10000  # Example vocabulary size
embed_dim = 128
num_class = 2  # Positive and Negative
model = SimpleSentimentClassifier(vocab_size, embed_dim, num_class)


**Training**

In [ ]:
from torch.utils.data import random_split

# Split the dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, data_loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(data_loader)

# Evaluation loop
def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

    accuracy = correct_preds / total_preds
    return total_loss / len(data_loader), accuracy



In [ ]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')


Epoch [1/10], Train Loss: 0.0003, Test Loss: 4.4179, Test Accuracy: 0.5164
Epoch [2/10], Train Loss: 0.0002, Test Loss: 4.5402, Test Accuracy: 0.5152
Epoch [3/10], Train Loss: 0.0002, Test Loss: 4.6654, Test Accuracy: 0.5177
Epoch [4/10], Train Loss: 0.0001, Test Loss: 4.7799, Test Accuracy: 0.5164
Epoch [5/10], Train Loss: 0.0001, Test Loss: 4.8971, Test Accuracy: 0.5164
Epoch [6/10], Train Loss: 0.0001, Test Loss: 5.0074, Test Accuracy: 0.5140
Epoch [7/10], Train Loss: 0.0001, Test Loss: 5.1201, Test Accuracy: 0.5128
Epoch [8/10], Train Loss: 0.0000, Test Loss: 5.2357, Test Accuracy: 0.5128
Epoch [9/10], Train Loss: 0.0000, Test Loss: 5.3421, Test Accuracy: 0.5140
Epoch [10/10], Train Loss: 0.0000, Test Loss: 5.4462, Test Accuracy: 0.5140
